In [3]:
import requests
from bs4 import BeautifulSoup

In [25]:
url = "https://internshala.com/internships/work-from-home-data-science,machine-learning-internships/"

In [5]:
req = requests.get(url)

In [9]:
soup = BeautifulSoup(req.content, "html.parser")

In [18]:
total_pages = int(soup.find(id='total_pages').text.strip())
total_pages

2

In [20]:
range(total_pages)

range(0, 2)

In [28]:
for page in range(total_pages):
    pg_url = url
    pg_url += f"page-{page+1}/"
    # print(pg_url)
    req_pg = requests.get(pg_url)
    soup = BeautifulSoup(req_pg.content, "html.parser")
    

https://internshala.com/internships/work-from-home-data-science,machine-learning-internships/page-1/
https://internshala.com/internships/work-from-home-data-science,machine-learning-internships/page-2/


In [130]:
pg_url = "https://internshala.com/internships/work-from-home-data-science,machine-learning-internships/page-2/"
req_pg = requests.get(pg_url)
soup = BeautifulSoup(req_pg.content, "html.parser")

In [131]:
internships = soup.find_all(class_ = 'heading_4_5 profile')

In [132]:
internships[2].find('a', href=True)["href"]

'/internship/detail/artificial-intelligence-ai-python-development-work-from-home-job-internship-at-scure-infosec1655700102'

In [165]:
sub_url = 'https://internshala.com' + internships[2].find('a', href=True)["href"]
sub_url = "https://internshala.com/internship/detail/market-research-analytics-internship-in-multiple-locations-at-justdial-limited1657168942"

In [166]:
req_sub_pg = requests.get(sub_url)
sub_soup = BeautifulSoup(req_sub_pg.content, "html.parser")

In [167]:
sub_soup.find(class_ = 'profile_on_detail_page').text.strip()

'Market Research (Analytics)'

In [168]:
sub_soup.find(class_ = 'heading_6 company_name').find('a').text.strip()

'Justdial Limited'

In [169]:
[i.text.strip() for i in sub_soup.find_all(class_ = 'location_link')]

['Delhi', 'Bangalore', 'Mumbai']

In [170]:
info = sub_soup.find(class_ = 'internship_other_details_container')
other_details = info.find_all(class_ = 'item_body')

In [171]:
other_details[1].text.strip()

'2 Months'

In [172]:
other_details[2].text.strip()

'2000 /month +  Incentives'

In [173]:
other_details[3].text.strip() # apply by

"21 Jul' 22"

In [174]:
sub_soup.find(class_ = 'applications_message').text.strip()

'132 applicants'

In [163]:
skills_raw = sub_soup.find(class_ = 'heading_5_5',string = 'Skill(s) required')

In [58]:
skills_raw.find_all(class_ = 'round_tabs')

[<span class="round_tabs">Financial literacy</span>,
 <span class="round_tabs">Labour laws</span>,
 <span class="round_tabs">MS-Excel</span>,
 <span class="round_tabs">MS-PowerPoint</span>]

In [141]:
[i.text.strip() for i in skills_raw.find_all(class_ = 'round_tabs')]

['Financial literacy', 'Labour laws', 'MS-Excel', 'MS-PowerPoint']

In [61]:
perks_raw = sub_soup.find(class_ = 'heading_5_5',string = 'Perks')
perks_raw = perks_raw.findNext(class_ = 'round_tabs_container')
[i.text.strip() for i in perks_raw.find_all(class_ = 'round_tabs')]

['Certificate', 'Letter of recommendation', 'Job offer']

In [63]:
sub_soup.find_all(class_='text-container')[-1].text.strip()

'4'

In [64]:
url

'https://internshala.com/internships/work-from-home-data-science,machine-learning-internships/'

In [86]:
def transform_categories(category_list):
    final_params = []
    for category in category_list:
        category = category.strip().lower()
        category = category.replace(' ','-')
        final_params.append(category)
    final_params = ','.join(final_params)
    final_params+='-internship'
    return final_params

In [87]:
transform_categories(['Data Science', 'Artificial Intelligence (AI)'])
# 

'data-science,artificial-intelligence-(ai)-internship'

In [75]:
list(map(str,input("Enter different categories separated by commas* (Required)\n").split(',')))

['Data Science', ' Machine Learning']

In [96]:
def transform_locations(location_list):
    final_params = []
    for location in location_list:
        location = location.strip().lower()
        location = location.replace(' ','-')
        final_params.append(location)
    final_params = ','.join(final_params)
    final_params ='-in-'+final_params

    return final_params

In [97]:
transform_locations(["Delhi", "Mumbai"])

'-in-delhi,mumbai'

In [98]:
base_url = "https://internshala.com/internships/"

In [142]:
input = {
    "category": ["Machine Learning"],
    "wfh": "yes",
    "location": []
}

In [143]:
import pandas as pd
df = pd.DataFrame()

In [178]:
df = pd.concat([df, pd.DataFrame([input])], ignore_index=True)

In [179]:
df

,category,wfh,location
0,"[Data Science, Machine Learning]",yes,"[Bangalore, Delhi]"
1,"[Data Science, Machine Learning]",yes,"[Bangalore, Delhi]"


In [12]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("notebooks"), '..')))
from src.web_scraper.scraper_functions import format_categories, format_locations

In [40]:
# from scraper_functions import format_categories, format_locations
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

class TooManyPages(Exception):
    def __init__(self, message="Please narrow search with filters, too many pages!"):
        self.message = message
        super().__init__(self.message)


def scraper(args_dict):

    base_url = "https://internshala.com/internships/"
    cat = args_dict["category"]
    wfh = args_dict["wfh"]
    loc = args_dict["location"]

    if wfh == "yes":
        base_url += "work-from-home-"
        wfh_status = True
        base_url += format_categories(cat, wfh_status)
    elif wfh == "no":
        wfh_status = False
        base_url += format_categories(cat, wfh_status)

    if len(loc) > 0:
        base_url += format_locations(loc)

    # base_url = "https://internshala.com/internships/data-science-internship-in-bangalore,delhi/early-applicant-25/"
    print(base_url)
    req = requests.get(base_url)
    soup = BeautifulSoup(req.content, "html.parser")
    total_pages = int(soup.find(id='total_pages').text.strip())

    if total_pages > 15:
        raise TooManyPages
    
    else:

        main_df = pd.DataFrame()

        for page in range(total_pages):
            pg_url = base_url
            pg_url += f"/page-{page+1}/"
            # print(pg_url)
            req_pg = requests.get(pg_url)
            soup = BeautifulSoup(req_pg.content, "html.parser")
            internships = soup.find_all(class_ = 'heading_4_5 profile')
            for internship in tqdm(internships):
                sub_info = {}
                sub_url = "https://internshala.com"
                sub_url += internship.find('a', href=True)["href"]
                req_sub_pg = requests.get(sub_url)
                sub_soup = BeautifulSoup(req_sub_pg.content, "html.parser")
                sub_info["title"] = sub_soup.find(class_ = 'profile_on_detail_page').text.strip()
                sub_info["company"] = sub_soup.find(class_ = 'heading_6 company_name').find('a').text.strip()
                sub_info["location"] = [i.text.strip() for i in sub_soup.find_all(class_ = 'location_link')]
                info = sub_soup.find(class_ = 'internship_other_details_container')
                other_details = info.find_all(class_ = 'item_body')
                sub_info["duration"] = other_details[1].text.strip()
                sub_info["stipend"] = other_details[2].text.strip()
                sub_info["apply_by"] = other_details[3].text.strip()
                sub_info["applicants"] = sub_soup.find(class_ = 'applications_message').text.strip()
                skills_raw = sub_soup.find(class_ = 'heading_5_5',string = 'Skill(s) required')
                try:
                    skills_raw = skills_raw.findNext(class_ = 'round_tabs_container')
                    skills_raw.find_all(class_ = 'round_tabs')
                    sub_info["skills"] = [i.text.strip() for i in skills_raw.find_all(class_ = 'round_tabs')]
                except(AttributeError):
                    sub_info["skills"] = []
                try:
                    perks_raw = sub_soup.find(class_ = 'heading_5_5',string = 'Perks')
                    perks_raw = perks_raw.findNext(class_ = 'round_tabs_container')
                    sub_info["perks"] = [i.text.strip() for i in perks_raw.find_all(class_ = 'round_tabs')]
                except(AttributeError):
                    sub_info["perks"] = []
                sub_info["openings"] = sub_soup.find_all(class_='text-container')[-1].text.strip()
                sub_info["url"] = sub_url
                sub_df = pd.DataFrame([sub_info])
                main_df = pd.concat([main_df, sub_df], ignore_index=True)

        return main_df.to_csv("raw_scraped.csv", index=False)

# scraper({
#     "category": ["Machine Learning"],
#     "wfh": "yes",
#     "location": []
# })

In [49]:
args_dict = {
    "category": ["data science", "web development"],
    "wfh": "yes",
    "location": ["bangalore"]
}
scraper(args_dict)

https://internshala.com/internships/work-from-home-data-science,web-development-internships-in-bangalore


100%|██████████| 25/25 [00:16<00:00,  1.49it/s]


In [255]:
df = pd.read_csv("../data/raw/raw_scraped.csv")
df.head()

,title,company,location,duration,stipend,apply_by,applicants,skills,perks,openings,url
0,Market Research (Analytics),Justdial Limited,"['Delhi', 'Bangalore', 'Mumbai']",2 Months,2000 /month + Incentives,21 Jul' 22,132 applicants,"['MS-Excel', 'MS-Office', 'MS-PowerPoint', 'Re...","['Certificate', 'Flexible work hours']",5,https://internshala.com/internship/detail/mark...
1,Web Development,IIT Bombay,['Work From Home'],6 Months,2000-5000 /month,20 Jul' 22,187 applicants,"['Django', 'MongoDB']","['Certificate', 'Letter of recommendation', 'F...",2,https://internshala.com/internship/detail/web-...
2,WordPress Development,NoBroker Technologies Solutions Private Limited,['Bangalore'],3 Months,8000 /month,19 Jul' 22,47 applicants,"['CSS', 'HTML', 'Java', 'JavaFX', 'JavaScript'...","['Certificate', 'Letter of recommendation', 'F...",4,https://internshala.com/internship/detail/word...
3,WordPress Web Development,Mechatron Robotics,['Work From Home'],6 Months,5000 /month,26 Jul' 22,Be an early applicant,['WordPress'],"['Certificate', 'Letter of recommendation']",2,https://internshala.com/internship/detail/word...
4,WordPress Development,Akeshya,['Work From Home'],3 Months,4000 /month + Incentives,26 Jul' 22,66 applicants,"['CSS', 'HTML', 'WordPress']","['Certificate', 'Letter of recommendation', 'F...",4,https://internshala.com/internship/detail/word...


In [196]:
df.loc[0, "duration"] = "2 weeks"

In [210]:
df["duration"] = df["duration"].str.replace("Months", "")
df["duration"] = df["duration"].str.replace("Month", "")
df["duration"] = df["duration"].apply(lambda x: float(x.strip()) if x.strip().isnumeric() else np.NaN)
# duration fixed, failing cols remove while validation only

In [256]:
df

,title,company,location,duration,stipend,apply_by,applicants,skills,perks,openings,url
0,Market Research (Analytics),Justdial Limited,"['Delhi', 'Bangalore', 'Mumbai']",2 Months,2000 /month + Incentives,21 Jul' 22,132 applicants,"['MS-Excel', 'MS-Office', 'MS-PowerPoint', 'Re...","['Certificate', 'Flexible work hours']",5,https://internshala.com/internship/detail/mark...
1,Web Development,IIT Bombay,['Work From Home'],6 Months,2000-5000 /month,20 Jul' 22,187 applicants,"['Django', 'MongoDB']","['Certificate', 'Letter of recommendation', 'F...",2,https://internshala.com/internship/detail/web-...
2,WordPress Development,NoBroker Technologies Solutions Private Limited,['Bangalore'],3 Months,8000 /month,19 Jul' 22,47 applicants,"['CSS', 'HTML', 'Java', 'JavaFX', 'JavaScript'...","['Certificate', 'Letter of recommendation', 'F...",4,https://internshala.com/internship/detail/word...
3,WordPress Web Development,Mechatron Robotics,['Work From Home'],6 Months,5000 /month,26 Jul' 22,Be an early applicant,['WordPress'],"['Certificate', 'Letter of recommendation']",2,https://internshala.com/internship/detail/word...
4,WordPress Development,Akeshya,['Work From Home'],3 Months,4000 /month + Incentives,26 Jul' 22,66 applicants,"['CSS', 'HTML', 'WordPress']","['Certificate', 'Letter of recommendation', 'F...",4,https://internshala.com/internship/detail/word...
...,...,...,...,...,...,...,...,...,...,...,...
540,Backend Development,Cognitio Eduventures Private Limited,['Work From Home'],4 Months,15000-20000 /month,20 Jul' 22,290 applicants,"['Amazon Web Server (AWS)', 'Apache Cassandra'...","['Certificate', 'Letter of recommendation', 'F...",3,https://internshala.com/internship/detail/back...
541,Quantitative Research,AR Quants,['Work From Home'],3 Months,1000 /month,15 Jul' 22,78 applicants,"['C++ Programming', 'Data Structures', 'Invest...",[],5,https://internshala.com/internship/detail/quan...
542,Backend Developement,Messold Technologies,['Work From Home'],6 Months,6000-15000 /month,19 Jul' 22,1000+ applicants,"['Bootstrap', 'CSS', 'HTML', 'JavaScript', 'jQ...","['Certificate', 'Letter of recommendation', '5...",4,https://internshala.com/internship/detail/back...
543,Front End Development (Shopify Website (Liquid)),Messold Technologies,['Work From Home'],6 Months,6000-15000 /month,19 Jul' 22,1000+ applicants,"['CSS', 'HTML', 'JavaScript', 'jQuery']","['Certificate', 'Letter of recommendation', '5...",2,https://internshala.com/internship/detail/fron...


In [257]:
df["stipend"] = df["stipend"].apply(lambda x: x if "month" in x else np.nan)

In [258]:
df["stipend"].isna().sum()

25

In [259]:
df["stipend"] = df["stipend"].dropna().apply(lambda x: x.split(" ")[0])

In [260]:
df["stipend"] = df["stipend"].str.replace("Unpaid", "0")

In [261]:
import numpy as np

In [262]:
def clean_stipend(row):
    try:
        if len(row.split("-")) == 2:
            return (float(row.split("-")[0]) + float(row.split("-")[0]))/2  #average
        else:
            try:
                return float(row)
            except:
                return np.nan
    except:
        np.nan

In [263]:
df["stipend"] = df["stipend"].apply(clean_stipend)

In [264]:
df["stipend"].isna().sum()

26

In [181]:
to_drop = []
to_drop += [int(i) for i in df[df["stipend"].isna()].index]
df.drop(to_drop, axis=0) ## log these rows

,title,company,location,duration,stipend,apply_by,applicants,skills,perks,openings,url
0,Market Research (Analytics),Justdial Limited,"['Delhi', 'Bangalore', 'Mumbai']",2,2000.0,21 Jul' 22,132 applicants,"['MS-Excel', 'MS-Office', 'MS-PowerPoint', 'Re...","['Certificate', 'Flexible work hours']",5,https://internshala.com/internship/detail/mark...
1,Web Development,IIT Bombay,['Work From Home'],6,2000.0,20 Jul' 22,187 applicants,"['Django', 'MongoDB']","['Certificate', 'Letter of recommendation', 'F...",2,https://internshala.com/internship/detail/web-...
2,WordPress Development,NoBroker Technologies Solutions Private Limited,['Bangalore'],3,8000.0,19 Jul' 22,47 applicants,"['CSS', 'HTML', 'Java', 'JavaFX', 'JavaScript'...","['Certificate', 'Letter of recommendation', 'F...",4,https://internshala.com/internship/detail/word...
3,WordPress Web Development,Mechatron Robotics,['Work From Home'],6,5000.0,26 Jul' 22,Be an early applicant,['WordPress'],"['Certificate', 'Letter of recommendation']",2,https://internshala.com/internship/detail/word...
4,WordPress Development,Akeshya,['Work From Home'],3,4000.0,26 Jul' 22,66 applicants,"['CSS', 'HTML', 'WordPress']","['Certificate', 'Letter of recommendation', 'F...",4,https://internshala.com/internship/detail/word...
...,...,...,...,...,...,...,...,...,...,...,...
540,Backend Development,Cognitio Eduventures Private Limited,['Work From Home'],4,15000.0,20 Jul' 22,290 applicants,"['Amazon Web Server (AWS)', 'Apache Cassandra'...","['Certificate', 'Letter of recommendation', 'F...",3,https://internshala.com/internship/detail/back...
541,Quantitative Research,AR Quants,['Work From Home'],3,1000.0,15 Jul' 22,78 applicants,"['C++ Programming', 'Data Structures', 'Invest...",[],5,https://internshala.com/internship/detail/quan...
542,Backend Developement,Messold Technologies,['Work From Home'],6,6000.0,19 Jul' 22,1000+ applicants,"['Bootstrap', 'CSS', 'HTML', 'JavaScript', 'jQ...","['Certificate', 'Letter of recommendation', '5...",4,https://internshala.com/internship/detail/back...
543,Front End Development (Shopify Website (Liquid)),Messold Technologies,['Work From Home'],6,6000.0,19 Jul' 22,1000+ applicants,"['CSS', 'HTML', 'JavaScript', 'jQuery']","['Certificate', 'Letter of recommendation', '5...",2,https://internshala.com/internship/detail/fron...


In [182]:
df["apply_by"] = pd.to_datetime(df["apply_by"], errors="raise")

In [183]:
df

,title,company,location,duration,stipend,apply_by,applicants,skills,perks,openings,url
0,Market Research (Analytics),Justdial Limited,"['Delhi', 'Bangalore', 'Mumbai']",2,2000.0,2022-07-21,132 applicants,"['MS-Excel', 'MS-Office', 'MS-PowerPoint', 'Re...","['Certificate', 'Flexible work hours']",5,https://internshala.com/internship/detail/mark...
1,Web Development,IIT Bombay,['Work From Home'],6,2000.0,2022-07-20,187 applicants,"['Django', 'MongoDB']","['Certificate', 'Letter of recommendation', 'F...",2,https://internshala.com/internship/detail/web-...
2,WordPress Development,NoBroker Technologies Solutions Private Limited,['Bangalore'],3,8000.0,2022-07-19,47 applicants,"['CSS', 'HTML', 'Java', 'JavaFX', 'JavaScript'...","['Certificate', 'Letter of recommendation', 'F...",4,https://internshala.com/internship/detail/word...
3,WordPress Web Development,Mechatron Robotics,['Work From Home'],6,5000.0,2022-07-26,Be an early applicant,['WordPress'],"['Certificate', 'Letter of recommendation']",2,https://internshala.com/internship/detail/word...
4,WordPress Development,Akeshya,['Work From Home'],3,4000.0,2022-07-26,66 applicants,"['CSS', 'HTML', 'WordPress']","['Certificate', 'Letter of recommendation', 'F...",4,https://internshala.com/internship/detail/word...
...,...,...,...,...,...,...,...,...,...,...,...
540,Backend Development,Cognitio Eduventures Private Limited,['Work From Home'],4,15000.0,2022-07-20,290 applicants,"['Amazon Web Server (AWS)', 'Apache Cassandra'...","['Certificate', 'Letter of recommendation', 'F...",3,https://internshala.com/internship/detail/back...
541,Quantitative Research,AR Quants,['Work From Home'],3,1000.0,2022-07-15,78 applicants,"['C++ Programming', 'Data Structures', 'Invest...",[],5,https://internshala.com/internship/detail/quan...
542,Backend Developement,Messold Technologies,['Work From Home'],6,6000.0,2022-07-19,1000+ applicants,"['Bootstrap', 'CSS', 'HTML', 'JavaScript', 'jQ...","['Certificate', 'Letter of recommendation', '5...",4,https://internshala.com/internship/detail/back...
543,Front End Development (Shopify Website (Liquid)),Messold Technologies,['Work From Home'],6,6000.0,2022-07-19,1000+ applicants,"['CSS', 'HTML', 'JavaScript', 'jQuery']","['Certificate', 'Letter of recommendation', '5...",2,https://internshala.com/internship/detail/fron...


In [184]:
df["applicants"] = df["applicants"].str.replace("Be an early applicant", "0")
df["applicants"] = df["applicants"].apply(lambda x: x.split(" ")[0])
df["applicants"] = df["applicants"].str.replace("+", "", regex=True)
df["applicants"] = df["applicants"].apply(lambda x: float(x.strip()) if x.strip().isnumeric() else np.NaN)

In [188]:
import ast
df["skills"] = df["skills"].apply(lambda x: ast.literal_eval(x))

In [189]:
df

,title,company,location,duration,stipend,apply_by,applicants,skills,perks,openings,url
0,Market Research (Analytics),Justdial Limited,"['Delhi', 'Bangalore', 'Mumbai']",2,2000.0,2022-07-21,132,"[MS-Excel, MS-Office, MS-PowerPoint, Research ...","['Certificate', 'Flexible work hours']",5,https://internshala.com/internship/detail/mark...
1,Web Development,IIT Bombay,['Work From Home'],6,2000.0,2022-07-20,187,"[Django, MongoDB]","['Certificate', 'Letter of recommendation', 'F...",2,https://internshala.com/internship/detail/web-...
2,WordPress Development,NoBroker Technologies Solutions Private Limited,['Bangalore'],3,8000.0,2022-07-19,47,"[CSS, HTML, Java, JavaFX, JavaScript, jQuery, ...","['Certificate', 'Letter of recommendation', 'F...",4,https://internshala.com/internship/detail/word...
3,WordPress Web Development,Mechatron Robotics,['Work From Home'],6,5000.0,2022-07-26,0,[WordPress],"['Certificate', 'Letter of recommendation']",2,https://internshala.com/internship/detail/word...
4,WordPress Development,Akeshya,['Work From Home'],3,4000.0,2022-07-26,66,"[CSS, HTML, WordPress]","['Certificate', 'Letter of recommendation', 'F...",4,https://internshala.com/internship/detail/word...
...,...,...,...,...,...,...,...,...,...,...,...
540,Backend Development,Cognitio Eduventures Private Limited,['Work From Home'],4,15000.0,2022-07-20,290,"[Amazon Web Server (AWS), Apache Cassandra, Ja...","['Certificate', 'Letter of recommendation', 'F...",3,https://internshala.com/internship/detail/back...
541,Quantitative Research,AR Quants,['Work From Home'],3,1000.0,2022-07-15,78,"[C++ Programming, Data Structures, Investing, ...",[],5,https://internshala.com/internship/detail/quan...
542,Backend Developement,Messold Technologies,['Work From Home'],6,6000.0,2022-07-19,1000,"[Bootstrap, CSS, HTML, JavaScript, jQuery]","['Certificate', 'Letter of recommendation', '5...",4,https://internshala.com/internship/detail/back...
543,Front End Development (Shopify Website (Liquid)),Messold Technologies,['Work From Home'],6,6000.0,2022-07-19,1000,"[CSS, HTML, JavaScript, jQuery]","['Certificate', 'Letter of recommendation', '5...",2,https://internshala.com/internship/detail/fron...


In [190]:
df["skills"][0]

['MS-Excel', 'MS-Office', 'MS-PowerPoint', 'Research and Analytics']

In [191]:
from collections import Counter
import operator

s_list = []
for i in df["skills"]:
    s_list += i

skill_cnt = dict(Counter(s_list))
skill_cnt_dict = dict(sorted(skill_cnt.items(), key=operator.itemgetter(1),reverse=True))
skill_count_df = pd.DataFrame(skill_cnt_dict.items(), columns=['Skill', 'Count'])
skill_count_df[:10]

,Skill,Count
0,JavaScript,237
1,HTML,210
2,CSS,203
3,Node.js,156
4,ReactJS,154
5,MySQL,126
6,PHP,121
7,Python,94
8,Bootstrap,90
9,jQuery,85


In [192]:
df["perks"] = df["perks"].apply(lambda x: ast.literal_eval(x))
p_list = []
for i in df["perks"]:
    p_list += i

perk_cnt = dict(Counter(p_list))
perk_cnt_dict = dict(sorted(perk_cnt.items(), key=operator.itemgetter(1),reverse=True))
perk_count_df = pd.DataFrame(perk_cnt_dict.items(), columns=['Skill', 'Count'])
perk_count_df[:10]

,Skill,Count
0,Certificate,467
1,Letter of recommendation,366
2,Flexible work hours,343
3,5 days a week,219
4,Job offer,150
5,Informal dress code,45
6,Free snacks & beverages,30


In [215]:
df[df.isna().any(axis=1)].index.to_list()
# index of all rows that have any nulls... will validate bad data out and you can log that row

[0]

In [217]:
bad_data = [0, 1, 2, 4, 5]

In [220]:
df.loc[bad_data].to_json()

'{"title":{"0":"Market Research (Analytics)","1":"Web Development","2":"WordPress Development","4":"WordPress Development","5":"Web Development"},"company":{"0":"Justdial Limited","1":"IIT Bombay","2":"NoBroker Technologies Solutions Private Limited","4":"Akeshya","5":"SilverOakHealth"},"location":{"0":"[\'Delhi\', \'Bangalore\', \'Mumbai\']","1":"[\'Work From Home\']","2":"[\'Bangalore\']","4":"[\'Work From Home\']","5":"[\'Work From Home\']"},"duration":{"0":null,"1":6.0,"2":3.0,"4":3.0,"5":2.0},"stipend":{"0":"2000 \\/month +  Incentives","1":"2000-5000 \\/month","2":"8000 \\/month","4":"4000 \\/month +  Incentives","5":"10000 \\/month"},"apply_by":{"0":"21 Jul\' 22","1":"20 Jul\' 22","2":"19 Jul\' 22","4":"26 Jul\' 22","5":"26 Jul\' 22"},"applicants":{"0":"132 applicants","1":"187 applicants","2":"47 applicants","4":"66 applicants","5":"121 applicants"},"skills":{"0":"[\'MS-Excel\', \'MS-Office\', \'MS-PowerPoint\', \'Research and Analytics\']","1":"[\'Django\', \'MongoDB\']","2":"

In [ ]:
with open("bad_data_files.log", "w") as f:
    for file in bad_data_list:
        f.write(file + "\n")
    f.close()

In [572]:
import pandas as pd
import ast
df = pd.read_csv("..\data\processed\prp_data.csv", parse_dates=["apply_by"])
df["skills"] = df["skills"].apply(lambda x: ast.literal_eval(x))
df["perks"] = df["perks"].apply(lambda x: ast.literal_eval(x))
df["apply_by"] = df["apply_by"].astype("int")

In [574]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       533 non-null    object 
 1   company     533 non-null    object 
 2   location    533 non-null    object 
 3   duration    533 non-null    float64
 4   stipend     533 non-null    float64
 5   apply_by    533 non-null    int32  
 6   applicants  533 non-null    float64
 7   skills      533 non-null    object 
 8   perks       533 non-null    object 
 9   openings    533 non-null    int64  
 10  url         533 non-null    object 
dtypes: float64(3), int32(1), int64(1), object(6)
memory usage: 43.8+ KB


In [575]:
d_avg = round(df["duration"].mean())
d_min = round(df["duration"].min())
d_min_count = (df["duration"] == d_min).sum()
d_max = round(df["duration"].max())
d_max_count = (df["duration"] == d_max).sum()
print(f"Average duration is {d_avg} month(s)")
print(f"Minimum duration is {d_min} month(s) offered by {d_min_count} internship(s)")
print(f"Maximum duration is {d_max} month(s) offered by {d_max_count} internship(s)")

Average duration is 4 month(s)
Minimum duration is 1 month(s) offered by 16 internship(s)
Maximum duration is 6 month(s) offered by 228 internship(s)


In [580]:
s_avg = round(df["stipend"].mean())
s_max = round(df["stipend"].max())
s_max_count = (df["stipend"] == s_max).sum()
print(f"Average stipend is Rs.{s_avg} / month")
print(f"Maximum stipend is Rs.{s_max} / month offered by {s_max_count} internship(s)")

Average stipend is Rs.8635 / month
Maximum stipend is Rs.50000 / month offered by 1 internship(s)


In [581]:
least_days = df["apply_by"].sort_values()[0:1].tolist()[0]
least_days_internship_count = (df["apply_by"] == least_days).sum()
print(f"Hurry up! {least_days_internship_count} internships expiring in {least_days} day(s)!")
# add if else for today and tommorrow

Hurry up! 35 internships expiring in -1 day(s)!


In [582]:
early_appl_stage = (df["applicants"] == 0).sum()
thousand_plus = (df["applicants"] == 1000).sum()
print(f"{early_appl_stage} internships are in early application stage, apply fast !!!")
print(f"{thousand_plus} internships have 1000+ applicants, not very likely to receive response...")

50 internships are in early application stage, apply fast !!!
6 internships have 1000+ applicants, not very likely to receive response...


In [583]:
applicants_srs = df[(df["applicants"] != 0) & (df["applicants"] != 1000)]["applicants"]
a_avg = round(applicants_srs.mean())
a_min = round(applicants_srs.min())
a_max = round(applicants_srs.max())
print(f"Average applicants is {a_avg} / internship")

Average applicants is 162 / internship


In [584]:
from collections import Counter
import operator

s_list = []
for i in df["skills"]:
    s_list += i

skill_cnt = dict(Counter(s_list))
skill_cnt_dict = dict(sorted(skill_cnt.items(), key=operator.itemgetter(1),reverse=True))
skill_count_df = pd.DataFrame(skill_cnt_dict.items(), columns=['Skill', 'Count'])
skill_count_df[:10]
# top 10 skills highest in demand

,Skill,Count
0,JavaScript,231
1,HTML,204
2,CSS,200
3,ReactJS,152
4,Node.js,151
5,MySQL,122
6,PHP,117
7,Python,91
8,Bootstrap,88
9,jQuery,84


In [585]:
p_list = []
for i in df["perks"]:
    p_list += i

perk_cnt = dict(Counter(p_list))
perk_cnt_dict = dict(sorted(perk_cnt.items(), key=operator.itemgetter(1),reverse=True))
perk_count_df = pd.DataFrame(perk_cnt_dict.items(), columns=['Skill', 'Count'])
perk_count_df[:5]
# top 5 perks most commonly on offer

,Skill,Count
0,Certificate,458
1,Letter of recommendation,360
2,Flexible work hours,336
3,5 days a week,215
4,Job offer,147


In [586]:
o_avg = round(df["openings"].mean())
o_max = round(df["openings"].max())
print(f"Average openings is {o_avg} / internship")
print(f"Maximum openings in an internship is {o_max}")

Average openings is 3 / internship
Maximum openings in an internship is 100


In [587]:
df.head()

,title,company,location,duration,stipend,apply_by,applicants,skills,perks,openings,url
0,Market Research (Analytics),Justdial Limited,"['Delhi', 'Bangalore', 'Mumbai']",2.0,2000.0,7,132.0,"[MS-Excel, MS-Office, MS-PowerPoint, Research ...","[Certificate, Flexible work hours]",5,https://internshala.com/internship/detail/mark...
1,Web Development,IIT Bombay,['Work From Home'],6.0,2000.0,6,187.0,"[Django, MongoDB]","[Certificate, Letter of recommendation, Flexib...",2,https://internshala.com/internship/detail/web-...
2,WordPress Development,NoBroker Technologies Solutions Private Limited,['Bangalore'],3.0,8000.0,5,47.0,"[CSS, HTML, Java, JavaFX, JavaScript, jQuery, ...","[Certificate, Letter of recommendation, Flexib...",4,https://internshala.com/internship/detail/word...
3,WordPress Web Development,Mechatron Robotics,['Work From Home'],6.0,5000.0,12,0.0,[WordPress],"[Certificate, Letter of recommendation]",2,https://internshala.com/internship/detail/word...
4,WordPress Development,Akeshya,['Work From Home'],3.0,4000.0,12,66.0,"[CSS, HTML, WordPress]","[Certificate, Letter of recommendation, Flexib...",4,https://internshala.com/internship/detail/word...


In [402]:
list(np.where(df["skills"].astype("str").str.lower().apply(lambda x: "css" in x))[0])

In [362]:
(df["skills"].astype("str").str.lower().apply(lambda x: "css" in x))

0      False
1      False
2       True
3      False
4       True
       ...  
514    False
515    False
516     True
517     True
518     True
Name: skills, Length: 519, dtype: bool

In [393]:
a = [3,4,5]
b = [4,5,6]
c = []

In [398]:
c += a
c += b
list(set(c))

[3, 4, 5, 6]

In [441]:
def get_skills_intersection_index(skills_list):
    intersection_skill_row_index = list(np.where(df["skills"].astype("str").str.lower().apply(lambda x: skills_list[0].lower() in x))[0])
    for skill in skills_list:
        sub_index = list(np.where(df["skills"].astype("str").str.lower().apply(lambda x: skill.lower() in x))[0])
        intersection_skill_row_index = list(set(intersection_skill_row_index) & set(sub_index))
    return intersection_skill_row_index

In [476]:
skills_list = ["css", "html", "python", "sql", "php", "mongodb", "tableau", "blabla", "werwer", "rwrwe"] 
# add a spelling validation for each skill before passing
intersection = get_skills_intersection_index(skills_list)

In [477]:
while not len(intersection) > 0:
    if len(skills_list) > 0:
        skills_list = skills_list[:-1]
        intersection = get_skills_intersection_index(skills_list)
    else:
        break
print(skills_list)
if len(intersection) > 0:
    print(df.loc[intersection])
else:
    print("Skills not found!") # this can never happen if u validate the entries as above

['css', 'html', 'python', 'sql', 'php']
                   title                      company            location  \
361  Backend Development                     One Root       ['Bangalore']   
467      Web Development                Legal Suvidha  ['Work From Home']   
446      Web Development  Wildly Tech Private Limited       ['Bangalore']   

     duration  stipend  apply_by  applicants  \
361       6.0   8000.0         2         0.0   
467       4.0   4500.0        -1       120.0   
446       6.0  20000.0        12        28.0   

                                                skills  \
361  [CSS, Git, HTML, JavaScript, Node.js, PHP, Pyt...   
467  [Bootstrap, CSS, HTML, Java, JavaFX, JavaScrip...   
446  [Bootstrap, CSS, HTML, JavaScript, jQuery, MyS...   

                                                 perks  openings  \
361  [Certificate, Letter of recommendation, Flexib...         1   
467  [Certificate, Letter of recommendation, Flexib...         2   
446                  

In [478]:
df.loc[intersection]

,title,company,location,duration,stipend,apply_by,applicants,skills,perks,openings,url
361,Backend Development,One Root,['Bangalore'],6.0,8000.0,2,0.0,"[CSS, Git, HTML, JavaScript, Node.js, PHP, Pyt...","[Certificate, Letter of recommendation, Flexib...",1,https://internshala.com/internship/detail/back...
467,Web Development,Legal Suvidha,['Work From Home'],4.0,4500.0,-1,120.0,"[Bootstrap, CSS, HTML, Java, JavaFX, JavaScrip...","[Certificate, Letter of recommendation, Flexib...",2,https://internshala.com/internship/detail/web-...
446,Web Development,Wildly Tech Private Limited,['Bangalore'],6.0,20000.0,12,28.0,"[Bootstrap, CSS, HTML, JavaScript, jQuery, MyS...",[Letter of recommendation],1,https://internshala.com/internship/detail/web-...


In [501]:
top10_result_index = list(df.loc[intersection]["duration"].apply(lambda x: abs(x-4)).sort_values()[:10].index)

In [503]:
recommended_df = df.loc[top10_result_index].sort_values(["applicants"])

In [504]:
recommended_df

,title,company,location,duration,stipend,apply_by,applicants,skills,perks,openings,url
361,Backend Development,One Root,['Bangalore'],6.0,8000.0,2,0.0,"[CSS, Git, HTML, JavaScript, Node.js, PHP, Pyt...","[Certificate, Letter of recommendation, Flexib...",1,https://internshala.com/internship/detail/back...
446,Web Development,Wildly Tech Private Limited,['Bangalore'],6.0,20000.0,12,28.0,"[Bootstrap, CSS, HTML, JavaScript, jQuery, MyS...",[Letter of recommendation],1,https://internshala.com/internship/detail/web-...
467,Web Development,Legal Suvidha,['Work From Home'],4.0,4500.0,-1,120.0,"[Bootstrap, CSS, HTML, Java, JavaFX, JavaScrip...","[Certificate, Letter of recommendation, Flexib...",2,https://internshala.com/internship/detail/web-...


In [588]:
all_skills = []
for skills in df["skills"]:
    all_skills += skills
all_skills = list(set(all_skills))

In [589]:
# user entered these values lets say
skills_list = ["css", "html", "python", "sql", "php", "mongodb", "tableau"]
duration = "54"

In [590]:
# validate it
def validate_skills_list(skills_list):
    condition = 0
    for skill in skills_list:
        if not skill.lower() in [i.lower() for i in all_skills]:
            condition = 1
            break
    if condition == 1:
        raise AttributeError("Skill entered not in list of skills.")
    elif condition == 0:
        return skills_list
    
def validate_duration(duration):
    try:
        return int(duration)
    except:
        raise AttributeError("Enter number of months as integer in duration.")

In [591]:
def get_skills_intersection_index(skills_list):
    intersection_skill_row_index = list(np.where(df["skills"].astype("str").str.lower().apply(lambda x: skills_list[0].lower() in x))[0])
    for skill in skills_list:
        sub_index = list(np.where(df["skills"].astype("str").str.lower().apply(lambda x: skill.lower() in x))[0])
        intersection_skill_row_index = list(set(intersection_skill_row_index) & set(sub_index))
    return intersection_skill_row_index

def get_recommendation(skills_list, duration):
    skills_list = validate_skills_list(skills_list)
    duration = validate_duration(duration)
    intersection = get_skills_intersection_index(skills_list)
    while not len(intersection) > 0:
        if len(skills_list) > 0:
            skills_list = skills_list[:-1]
            intersection = get_skills_intersection_index(skills_list)
        else:
            break
    skill_intersect_df = df.loc[intersection]
    top10_result_index = list(skill_intersect_df["duration"].apply(lambda x: abs(x-duration)).sort_values()[:10].index)
    recommended_df = df.loc[top10_result_index].sort_values(["applicants"])
    return recommended_df

In [592]:
get_recommendation(skills_list, duration)

,title,company,location,duration,stipend,apply_by,applicants,skills,perks,openings,url
374,Backend Development,One Root,['Bangalore'],6.0,8000.0,2,0.0,"[CSS, Git, HTML, JavaScript, Node.js, PHP, Pyt...","[Certificate, Letter of recommendation, Flexib...",1,https://internshala.com/internship/detail/back...
460,Web Development,Wildly Tech Private Limited,['Bangalore'],6.0,20000.0,12,28.0,"[Bootstrap, CSS, HTML, JavaScript, jQuery, MyS...",[Letter of recommendation],1,https://internshala.com/internship/detail/web-...
481,Web Development,Legal Suvidha,['Work From Home'],4.0,4500.0,-1,120.0,"[Bootstrap, CSS, HTML, Java, JavaFX, JavaScrip...","[Certificate, Letter of recommendation, Flexib...",2,https://internshala.com/internship/detail/web-...


In [593]:
recommended_df

,title,company,location,duration,stipend,apply_by,applicants,skills,perks,openings,url
361,Backend Development,One Root,['Bangalore'],6.0,8000.0,2,0.0,"[CSS, Git, HTML, JavaScript, Node.js, PHP, Pyt...","[Certificate, Letter of recommendation, Flexib...",1,https://internshala.com/internship/detail/back...
446,Web Development,Wildly Tech Private Limited,['Bangalore'],6.0,20000.0,12,28.0,"[Bootstrap, CSS, HTML, JavaScript, jQuery, MyS...",[Letter of recommendation],1,https://internshala.com/internship/detail/web-...
467,Web Development,Legal Suvidha,['Work From Home'],4.0,4500.0,-1,120.0,"[Bootstrap, CSS, HTML, Java, JavaFX, JavaScrip...","[Certificate, Letter of recommendation, Flexib...",2,https://internshala.com/internship/detail/web-...


In [594]:
abcd = {}

In [595]:
abcd["recommended_df"] = recommended_df

In [597]:
abcd["recommended_df"]

,title,company,location,duration,stipend,apply_by,applicants,skills,perks,openings,url
361,Backend Development,One Root,['Bangalore'],6.0,8000.0,2,0.0,"[CSS, Git, HTML, JavaScript, Node.js, PHP, Pyt...","[Certificate, Letter of recommendation, Flexib...",1,https://internshala.com/internship/detail/back...
446,Web Development,Wildly Tech Private Limited,['Bangalore'],6.0,20000.0,12,28.0,"[Bootstrap, CSS, HTML, JavaScript, jQuery, MyS...",[Letter of recommendation],1,https://internshala.com/internship/detail/web-...
467,Web Development,Legal Suvidha,['Work From Home'],4.0,4500.0,-1,120.0,"[Bootstrap, CSS, HTML, Java, JavaFX, JavaScrip...","[Certificate, Letter of recommendation, Flexib...",2,https://internshala.com/internship/detail/web-...
